**Task 07: Querying RDF(s)**

In [1]:
!pip install rdflib 
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2020-2021/master/Assignment4"

     |████████████████████████████████| 235kB 2.8MB/s 
     |████████████████████████████████| 51kB 4.5MB/s 


Read the RDF file

In [2]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.namespace_manager.bind('vcard', Namespace("http://www.w3.org/2001/vcard-rdf/3.0#"), override=False)
g.parse(github_storage+"/resources/example6.rdf", format="xml")

<Graph identifier=N45a320cf7fae4baa8cf662a259ab9a9a (<class 'rdflib.graph.Graph'>)>

In [3]:
from rdflib.plugins.sparql import prepareQuery
from rdflib import XSD
from rdflib import FOAF
NS = Namespace("http://somewhere#")
VCARD = Namespace("http://www.w3.org/2001/vcard-rdf/3.0#")

**TASK 7.1: List all subclasses of "Person" with RDFLib and SPARQL**

In [17]:
q71 = prepareQuery( '''
    SELECT ?s
    WHERE {
      ?s rdfs:subClassOf* ns:Person.
    } ''', 
    initNs = {'ns' : NS}
)

In [18]:
for r in g.query(q71):
  print(r)

(rdflib.term.URIRef('http://somewhere#Person'),)
(rdflib.term.URIRef('http://somewhere#Researcher'),)
(rdflib.term.URIRef('http://somewhere#PhDstudent'),)


In [11]:
for s in g.subjects( predicate= RDFS.subClassOf, object= NS.Person):
  for ss in g.subjects (predicate= RDFS.subClassOf, object= s):
    print(s,ss)

http://somewhere#Researcher http://somewhere#PhDstudent


**TASK 7.2: List all individuals of "Person" with RDFLib and SPARQL (remember the subClasses)**


In [19]:
q72 = prepareQuery( '''
    SELECT ?s
    WHERE {
     { 
      ?s a ns:Person.
     }  
     UNION
     {
      ?s a ?y.
      ?y rdfs:subClassOf ns:Person
     }   
    } ''', 
    initNs={'ns' : NS }
)

In [20]:
for r in g.query(q72):
  print(r)

(rdflib.term.URIRef('http://somewhere#JohnSmith'),)
(rdflib.term.URIRef('http://somewhere#SaraJones'),)
(rdflib.term.URIRef('http://somewhere#JaneSmith'),)


In [21]:
for s in g.subjects(predicate = RDF.type, object = NS.Person):
  print(s)
for x in g.subjects(predicate= RDFS.subClassOf, object= NS.Person):
  for s in g.subjects(predicate= RDF.type, object= x):
    print(s)


http://somewhere#JohnSmith
http://somewhere#SaraJones
http://somewhere#JaneSmith


**TASK 7.3: List all individuals of "Person" and all their properties including their class with RDFLib and SPARQL**


In [22]:
q73 = prepareQuery( '''
    SELECT ?s ?p
    WHERE {
      ?s a ns:Person.
      ?s ?p ?o
    } ''', 
     initNs = {'ns' : NS}
)

In [23]:
for r in g.query(q73):
  print(r)

(rdflib.term.URIRef('http://somewhere#JohnSmith'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'))
(rdflib.term.URIRef('http://somewhere#JohnSmith'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given'))
(rdflib.term.URIRef('http://somewhere#JohnSmith'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family'))
(rdflib.term.URIRef('http://somewhere#JohnSmith'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN'))
(rdflib.term.URIRef('http://somewhere#SaraJones'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given'))
(rdflib.term.URIRef('http://somewhere#SaraJones'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'))
(rdflib.term.URIRef('http://somewhere#SaraJones'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN'))
(rdflib.term.URIRef('http://somewhere#SaraJones'), rdflib.term.URIRef('http://somewhere#htt

In [24]:
for s in g.subjects(predicate= RDF.type, object= NS.Person):
  for s, p, o in g.triples((s, None, None)):
    print(s, p)

http://somewhere#JohnSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given
http://somewhere#SaraJones http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family


In [25]:
print(g.serialize(format="turtle").decode("UTF-8"))

@prefix ns: <http://somewhere#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix vcard-rdf: <http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ns:Person a rdfs:Class .

ns:PhDstudent a rdfs:Class ;
    rdfs:subClassOf ns:Researcher .

ns:Researcher a rdfs:Class ;
    rdfs:subClassOf ns:Person .

ns:University a rdfs:Class .

ns:JaneSmith a ns:Researcher ;
    vcard-rdf:FN "jane Smith"^^xsd:string ;
    vcard-rdf:Family "Smith"^^xsd:string ;
    vcard-rdf:Given "Jane"^^xsd:string .

ns:JohnSmith a ns:Person ;
    vcard-rdf:FN "John Smith"^^xsd:string ;
    vcard-rdf:Family "Smith"^^xsd:string ;
    vcard-rdf:Given "John"^^xsd:string .

ns:SaraJones a ns:Person ;
    vcard-rdf:FN "Sara Jones"^^xsd:string ;
    vcard-rdf:Family "Jones"^^xsd:string ;
    vcard-rdf:Given "Sara"^^xsd:string .

ns:UPM a ns:University .

vcard-rdf:FN a rdfs:Datatype ;
    rdfs:range xsd:string .

vcard-rdf:Family a rdfs:Property ;
   